# Chapter 1 - Introduction

Link to "m-file": http://people.maths.ox.ac.uk/trefethen/ExplODE/ode01.m

## Environment setup
Setup the local stuff - first you may need to change the Kernel, but not always.

Then setup the packages as needed. The stuff that's commented out should already be there from the build. Don't actually load that unless you need to.


In [ ]:
using Pkg
Pkg.status()

# Pkg.add("DifferentialEquations")
# Pkg.add("Plots")
# Pkg.add("DiffEqBase")
# Pkg.add("DiffEqOperators")
# Pkg.add("ModelingToolkit")
# Pkg.status()

using DifferentialEquations
using Plots
using DiffEqBase, DiffEqOperators, LinearAlgebra
using ModelingToolkit


# Excercise 1

_Local extrema of van der Pol oscillation._ In the van der Pol example
(1.2), you can find the local maxima with $max(y,'local')$, and similarly for minima
with $min$; you can find both minima and maxima at once with $minandmax$. How close is
the first local minimum value (at $t \approx 1.2$) to its asymptotic value for $t \to \infty$? Likewise for the first local maximum (at $t \approx 3.2$)?

Note, that in Julia I didn't see any way to make the minimum/maximum functions work like that, hence the ones I built, at least till I learn of a better way.


## From example 1.2

Van der Pol equation: $0.3u'' -(1-u^2)u' + u = 0$

Borrowing the code the examples Git Repo refered to in the _DifferentialEquations.jl_ docs (look for the Van der Pol examples):
https://github.com/SciML/DiffEqProblemLibrary.jl/blob/master/src/ode/ode_simple_nonlinear_prob.jl

We rewrite the Second Order Function as a system of First Order Functions (per Excercise 1.5):

$$ \frac{dy}{dt} = \mu ((1-x^2)y - x)$$

$$ \frac{dx}{dt} = y$$

Where $\mu = \frac 1 {0.3}$

so:


In [ ]:
@parameters t μ
@variables x(t) y(t)
D = Differential(t)
eqs = [D(y) ~ μ*((1-x^2)*y - x),
       D(x) ~ y]
de = ODESystem(eqs; name=:van_der_pol)
van = ODEFunction(de, [y, x], [μ], jac=true)
u0 = [0.0; 1.0]
vspan = (0.0, 20.0)
μ = 1/0.3
prob_ode_vanderpol = ODEProblem(van, u0, vspan, [μ])
vsol = solve(prob_ode_vanderpol, reltol=1e-8, abstol=1e-8)
plot(vsol, vars=(0, 2))

And get the minimas. First Load my utility routines.

In [ ]:
include("localminmax.jl")

In [ ]:
sigdigits = 6

result = [v[2] for v in vsol.u]
mins = localmins(result)
last_min = nothing
for cur_min in mins
    print("t = ", round(vsol.t[cur_min[1]], sigdigits=sigdigits),
          " u = ", round(cur_min[2], sigdigits=sigdigits))
    if last_min != nothing
        println("   Diff to last = ", round(vsol.t[cur_min[1]] - last_min, sigdigits=sigdigits))
    else
        println("")
    end
    last_min = vsol.t[cur_min[1]]
end
println("")

maxs = localmaxs(result)
last_max = nothing
for cur_max in maxs
    print("t = ", round(vsol.t[cur_max[1]], sigdigits=sigdigits),
          " u = ", round(cur_max[2], sigdigits=sigdigits))
    if last_max != nothing
        println("   Diff to last = ", round(vsol.t[cur_max[1]] - last_max, sigdigits=sigdigits))
    else
        println("")
    end
    last_max = vsol.t[cur_max[1]]
end
println("")


**To be updated** I have updated the code above, but not the discussion below, which is based on the used of _SecondOrderODEProblem()_, which didn't give as accurate results.


The minima at $t \approx 1.2: u'' = -1.93803$ which is $0.07$ off from the minima at $t = 5.2073: u'' = -2.0185$ which closely matches the following minima. However the maxima at $t \approx 3.2: u''=2.01843$ which has already converged to what we see in the following maxima to within 4 significant digits.

Unfortunately we are not getting the same numbers as in the book, they are the same to 3 significant digits, but the book has 5 and the period is certainly not the same to 5 significant digits.

# Exercise 1.2

_Classification of ODE problems._ Classify the following ODE problems
according to the FLASHI scheme.

1. $y' = sin(t) − y, t \in [0, 100], y(0) = 1$
    * FLaSHI
2. $y' = sin(t) − y^3, t \in [0, 100], y(0) = 1$
    * FlaSHI
3. (Nonlinear pendulum equation) $y'' = −sin(y), t \in [0, 10], y(0) = y(10) = 2$
    * fLASHI
4. (Advection-diffusion equation) $0.02y'' + y' + y = 0, t \in [0, 1], y(0) = 0, y(1) = 1$
    * fLASHI
5. (Airy equation) $0.02y'' − ty = 0, t \in [−5, 5], y(−5) = 1, y(5) = 0$
    * fLaSHi
6. (Harmonic oscillator) $u' = v, v' = −u, t \in [0, 100], u(0) = 1, v(0) = 0$
    * FLAsHI
7. $u' = u^2v, v' = −uv^2, t \in [0, 2], u(0) = 1, v(0) = 0$
    * FlAsHI
8. (Bessel equation) $t^2y'' + ty' + (t^2 − 4)y = 0, t \in [0, 8], y(0) = 0, y(8) = 1$
    * flaSHI
9. $0.1y'' + yy' = y, t \in [−1, 1], y(−1) = −2, y(1) = 1$
    * flASHi
10. (Lotka–Volterra equations) $u' = u(1−v), v' = v(u−1), t \in [0, 10], u(0) = v(0) = 1$
    * FlAsHi
11. (Blasius equation) $y''' + 0.5yy'' = 0, t \in [0, 10], y(0) = y'(0) = 0, y'(10) = 1$
    * flASHI

# Exercise 1.3

_Airy equation._ Plot the solution $y$ of the problem of Exercise 1.2(5)
and report its maximum value. Letting $k$ denote the coefficient $0.02$, do the same
with $k = 0.002$. Now make a plot of $max(y)$ as a function of $k$ for the values $k =
0.001, 0.002, . . . , 0.039, 0.040$. (We shall explore such effects in Chapter 6.)

(Airy equation) $0.02y'' − ty = 0, t \in [−5, 5], y(−5) = 1, y(5) = 0$

Rewriting for Julia package: $y'' = ty/0.02, t \in [-5, 5], y(-5) = 1, y(5) = 0$

In [ ]:
# airy(dy, y, p, t) = t*y / 0.02
airy(dy, y, p, t) = t*y  # To match Fig 7.2, page 78
airyspan = (-5.0, 5.0)
airyprob = SecondOrderODEProblem(airy, 1.0, 0.0, airyspan)
airysol = solve(airyprob, reltol=1e-8, abstol=1e-8)

plot(airysol)


This clearly doesn't match Fig 7.2 on page 78, so clearly I am missing something with the Julia $DifferentialEquations$ package interface. More work needed, and maybe that will fix my variation on Excercise 1.


# Exercise 1.4

_Solution with rapid transient._ Plot the solution y of the problem of
Exercise 1.2(9), and give its maximum slope $s = max_{t \in [−1,1]} y'(t)$. Do the same with
the coefficient $0.1$ reduced to $1/20$, $1/40$, and $1/80$.

# Exercise 1.5

_Reduction to first-order system._ (a) Show how the IVP of Exercise 1.2(6)
can be rewritten as a second-order scalar IVP involving just the dependent variable u.
What are the initial conditions for this IVP? (b) Conversely, show how the third-order
IVP of Exercise 1.2(11) can be rewritten as a first-order system involving three variables
$u$, $v$, and $w$. Any higher-order ODE problem can be rewritten as a first-order problem
like this, and numerical software for IVPs often requires the problem to be expressed
in first-order form.

# Exercise 1.6

How Chebfun represents functions. Chebfun normally represents solutions
$y(t)$ to ODEs by polynomial approximations, typically with an accuracy of about
10 digits, whose degree $n$ may be quite high. The polynomials can be interpreted as
interpolants through a sufficiently large number $n + 1$ of samples at Chebyshev points
defined by $t_j = cos(\pi j/n), 0 \le j \le n$ for $t \in [−1, 1]$, or linearly transplanted to a
different interval $[a, b]$. (a) Let $y$ be the computed solution of the problem (1.1). Execute
$length(y)$ to find the number $n + 1$ for this function, and $plot(y,'.-')$ to see
the associated Chebyshev points. (b) Do the same for the computed solution of the
van der Pol problem (1.2). Approximately speaking (say, to within 10%), how many
interpolation points are there on average per wavelength?

# Exercise 1.7

How Chebfun represents periodic functions. Chebfun also has a representation
for periodic functions that takes advantage of the periodicity, based on
trigonometric polynomials (i.e., Fourier series) rather than ordinary algebraic polynomials.
Periodic solutions arise naturally in ODEs with periodic coefficients (see Chapter
19). (a) Construct an ordinary chebfun for $f(t) = (1.1 − cos(\pi t))
−1, t \in [−1, 1]$
with the command $chebfun('1/(1.1-cos(pi*x))')$. What is its length? (b) How
does the length change if you use $chebfun('1/(1.1-cos(pi*x))','trig')$? (In an
appropriate limit, the ratio of the two lengths approaches $\pi/2$.)